In [42]:
# import necessary modules
import jax.numpy as jnp
# import numpy as np
import jax
jax.config.update("jax_enable_x64", True)
from astropy.io import fits
import os
import jaxopt
import matplotlib.pyplot as plt
from functools import partial

import load_data as ld
import optimise as opt
import scatters as opt_sc
import init_latents as il
import kfold_cv as kf
import jax.numpy as jnp
import numpy as np
import tqdm

from cannon import CannonModel 
import cannon as c
from cannon import Dataset 

# load the data
file_name = '-train-rgbs-new'
spectra_dir_path = '../spec/spectra-reference-stars-APOGEE-giants-ref/'
file_path = '../data/master-APOGEE-giants-train.fits'
spectra_data, label_data = ld.load_data(spectra_dir_path, file_path, file_name)


../spec/spectra-reference-stars-APOGEE-giants-ref/spectra_data-train-rgbs-new.dat
File already exists. Loading spectra data


24it [00:00, 627.90it/s]

Loaded data successfully


In [21]:
# load the data
file_name_test = '-test-rgbs'
spectra_dir_path_test = '../spec/spectra-reference-stars-APOGEE-giants-10K-ref/'
file_path_test = '../data/master-APOGEE-giants-test.fits'
spectra_data_test, label_data_test = ld.load_data(spectra_dir_path_test, file_path_test, file_name_test)

../spec/spectra-reference-stars-APOGEE-giants-10K-ref/spectra_data-test-rgbs.dat
File already exists. Loading spectra data


24it [00:00, 446.69it/s]

Loaded data successfully


In [23]:
wl = spectra_data['wl']
train_ID = label_data['ids']
train_flux = spectra_data['fluxes']
train_ivar = spectra_data['fluxes_ivars']
train_label = label_data['labels']

test_ID = label_data_test['ids']
test_flux = spectra_data_test['fluxes']
test_ivar = spectra_data_test['fluxes_ivars']
test_label = label_data_test['labels']

In [24]:
ds = Dataset(wl, train_ID, train_flux, train_ivar, train_label, test_ID, test_flux, test_ivar, test_label)


Loading dataset


# Train: optimise the model to learn thetas and scatters

In [25]:
model = CannonModel(order=2)

In [26]:
# get the pivots and scales
fluxes_train = ds.flux_train
ivars_train = ds.ivar_train
labels_train = ds.label_train

pivots, scales = model.get_pivots_scales(labels_train)

In [27]:
# just to get the right size 
lvec_train = model.get_lvec(labels_train, pivots, scales)

# set up the data and params dictionaries
theta_array = jnp.ones((fluxes_train.shape[1], lvec_train.shape[1]))
params_train = {'theta': theta_array}
data_train = {'fluxes': fluxes_train, 'ivars':ivars_train, 'labels': labels_train}

print(jnp.shape(params_train['theta']), jnp.shape(data_train['fluxes']), jnp.shape(data_train['ivars']),jnp.shape(data_train['labels']))


(8575, 91) (5000, 8575) (5000, 8575) (5000, 12)


In [28]:
train_model = jax.vmap(model.least_sq_solve_onewavelength, in_axes=( {'theta': 0}, {'fluxes': 1, 'ivars': 1, 'labels': None}, None, None)) # here the 0 and 1 in the theta, scatter, fluxes etc corresponds to the dimension of the matrix to loop over


In [29]:
thetas = train_model(params_train, data_train, pivots, scales)


In [30]:
train_synthetic = model.get_synthetic_spectra_onestar(thetas, data_train, pivots, scales)


In [50]:
savepath = '../sav/paper/'
name = '_49_L2regstrength1000_omega1-train-highsnrgiants'
zetas_test_fromfluxes = np.load(savepath+'zetas_test_fromflux_giants-withscatters'+str(name)+'.npy', allow_pickle=True)
labels_test_fromflux = np.load(savepath+'labels_test_fromflux_giants-withscatters'+str(name)+'.npy', allow_pickle=True)
spectra_test_fromflux = np.load(savepath+'spectra_test_fromflux_giants-withscatters'+str(name)+'.npy', allow_pickle=True)
# labels_test_fromflux = zetas_test_fromfluxes @ alphas.T
# spectra_test_fromflux = zetas_test_fromfluxes @ betas.T

savepath = '../sav/paper/'
name = '_P49_L2regstrength1000_omega1-train-highsnrgiants'
zetas_test_fromlabels = np.load(savepath+'zetas_test_fromlabels_giants-withscatters'+str(name)+'.npy', allow_pickle=True)
labels_test_fromlabels = np.load(savepath+'labels_test_fromlabels_giants-withscatters'+str(name)+'.npy', allow_pickle=True)
spectra_test_fromlabels = np.load(savepath+'spectra_test_fromlabels_giants-withscatters'+str(name)+'.npy', allow_pickle=True)


In [37]:
opt_ = []

for indx in tqdm.tqdm_notebook(range(len(test_label))):
    rand_labels_it = np.random.rand(test_label.shape[1])
    opt_.append(model.optimise_labels(thetas, ds.flux_test[indx:indx+1], ds.ivar_test[indx:indx+1], rand_labels_it[None], pivots, scales))

/var/folders/rq/q7l_wtbx4sx5ky0yrvbk7fgh0000gq/T/ipykernel_65778/1446818195.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for indx in tqdm.tqdm_notebook(range(len(test_label))):


  0%|          | 0/10000 [00:00<?, ?it/s]

INFO: jaxopt.ZoomLineSearch: Iter: 30, Stepsize: 1.862645149230957e-09, Decrease error: 2.8839012578842654e-08, Curvature error: 1.548015602567305
INFO: jaxopt.ZoomLineSearch: Iter: 30, Stepsize: 1.862645149230957e-09, Decrease error: 2.1491027629783947e-08, Curvature error: 1.1548966958699922
INFO: jaxopt.ZoomLineSearch: Iter: 30, Stepsize: 3.32228547284578e-10, Decrease error: 5.449147110285341e-09, Curvature error: 1.640866629064032
INFO: jaxopt.ZoomLineSearch: Iter: 30, Stepsize: 1.862645149230957e-09, Decrease error: 4.152337612031006e-08, Curvature error: 2.2290679455506375
INFO: jaxopt.ZoomLineSearch: Iter: 30, Stepsize: 1.862645149230957e-09, Decrease error: 1.4822553546527083e-07, Curvature error: 7.959093154081776
INFO: jaxopt.ZoomLineSearch: Iter: 30, Stepsize: 1.101692803146198e-09, Decrease error: 5.463006614931575e-07, Curvature error: 49.590713166905104
INFO: jaxopt.ZoomLineSearch: Iter: 30, Stepsize: 1.862645149230957e-09, Decrease error: 7.12099509495037e-08, Curvature

In [38]:
labels_pred = np.squeeze([o.params['labels'] for o in opt_])


In [41]:
savepath = '../sav/paper/'
name = 'Cannon_trained'
np.save(savepath+str(name)+'thetas', jnp.array(thetas))
np.save(savepath+str(name)+'pivots_train', jnp.array(pivots))
np.save(savepath+str(name)+'scales_train', jnp.array(scales))
np.save(savepath+str(name)+'lvec_train', jnp.array(lvec_train))
np.save(savepath+str(name)+'labels-test', jnp.array(labels_pred))
